In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from MyEarlyStopping import MyEarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  
import pandas as pd
import timeit

In [3]:
All_data = ImageDataGenerator(preprocessing_function=preprocess_input)
All_data_generator = All_data.flow_from_directory('/data/home/cou/yuli5/Audio/online_data_adaption/Mfcc_Audio_folder_digital',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 30000 images belonging to 10 classes.


In [4]:
image_names = All_data_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = All_data_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=60)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [5]:
All_Inputs = [next(All_data_generator)[0][0] for _ in range(len(All_data_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
# image_no<=20 means every word repeat 20 times 
s_number = 5 # ALL_participant_class  the subject index

select_indexs_train = (image_no<=19)&(ALL_participant_class<=s_number)
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train][:,0:s_number]
Train_participant_uniform = Train_participant_class*0+1/s_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10

In [7]:
select_indexs_val = (image_no>29)&(image_no<=39)&(ALL_participant_class<=s_number)
Val_Inputs = All_Inputs[select_indexs_val]
Val_participant_class = All_participant_class[select_indexs_val][:,0:s_number]
Val_participant_uniform = Val_participant_class*0+1/s_number
Val_command_class = All_command_class[select_indexs_val]
Val_command_uniform = Val_command_class*0+1/10

In [8]:
select_indexs_test = (image_no>39)&(image_no<=49)&(ALL_participant_class<=s_number)
Test_Inputs = All_Inputs[select_indexs_test]
Test_participant_class = All_participant_class[select_indexs_test][:,0:s_number]
Test_participant_uniform = Test_participant_class*0+1/s_number
Test_command_class = All_command_class[select_indexs_test]
Test_command_uniform = Test_command_class*0+1/10

In [9]:
Perfomance = pd.read_csv('Performance_0915_Digit_Inputs.csv')

In [10]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


In [11]:
high_acc = 0
for run in range(0,10):
    
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(s_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])

    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])
    start = timeit.default_timer()
    history1 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)
   
    stop = timeit.default_timer()
    run_time = stop - start
    
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    participant_class = [i.split("/")[1].split("_")[1] for i in All_data_generator.filenames]
    participant_class = np.array(participant_class)[select_indexs_test]
    participant_class = np.array(list(map(int, participant_class)))
    acc_s = round(sum(x == y for x, y in zip(participant_class, predicted_classes)) / len(participant_class),4)
    
    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.array([i.split("/")[0] for i in All_data_generator.filenames])
    true_classes = np.array(list(map(int, true_classes)))[select_indexs_test]
    acc_c = round(sum(x == y for x, y in zip(true_classes, predicted_classes)) / len(true_classes),4)
    overall_acc = acc_c + acc_s

    Perfomance = Perfomance.append({'Feature_Extract': "ResNet",'Inputs': "MFCC",'Time':run_time,
                                    'Partcp_Acc':acc_s,'Command_Acc':acc_c}, ignore_index=True)
    
    if high_acc < overall_acc:
        resnet_model.save('Digital_data_Initial_group_model_0915_MFCC.h5')
        high_acc = overall_acc
        best_index = run
        
    del resnet_model
    run = run + 1

Epoch 1/10
16/16 [==============================] - 5s 292ms/step - loss: 3.9230 - participant_output_loss: 1.6120 - command_output_loss: 2.3051 - command_output_1_loss: 0.0012 - participant_output_1_loss: 0.0047 - participant_output_accuracy: 0.2640 - command_output_accuracy: 0.2070 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1180 - val_loss: 3.6388 - val_participant_output_loss: 1.4314 - val_command_output_loss: 2.2072 - val_command_output_1_loss: 1.5059e-04 - val_participant_output_1_loss: 7.5143e-05 - val_participant_output_accuracy: 0.5640 - val_command_output_accuracy: 0.3500 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.4640
Epoch 2/10
16/16 [==============================] - 3s 171ms/step - loss: 3.5110 - participant_output_loss: 1.3333 - command_output_loss: 2.1775 - command_output_1_loss: 8.5256e-05 - participant_output_1_loss: 9.7210e-05 - participant_output_accuracy: 0.6300 - command_output_accuracy: 0.4770

16/16 [==============================] - 2s 145ms/step - loss: 1.8560 - participant_output_loss: 0.2480 - command_output_loss: 1.6079 - command_output_1_loss: 3.2803e-05 - participant_output_1_loss: 1.4335e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0070 - participant_output_1_accuracy: 0.2850 - val_loss: 1.9713 - val_participant_output_loss: 0.3718 - val_command_output_loss: 1.5994 - val_command_output_1_loss: 2.0280e-05 - val_participant_output_1_loss: 1.5157e-05 - val_participant_output_accuracy: 0.9640 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2720
Epoch 3/10
16/16 [==============================] - 2s 144ms/step - loss: 1.7690 - participant_output_loss: 0.2176 - command_output_loss: 1.5513 - command_output_1_loss: 4.5733e-05 - participant_output_1_loss: 1.3916e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - comm

Epoch 4/10
16/16 [==============================] - 3s 174ms/step - loss: 3.0722 - participant_output_loss: 1.1009 - command_output_loss: 1.9712 - command_output_1_loss: 5.5842e-05 - participant_output_1_loss: 2.5117e-05 - participant_output_accuracy: 0.8440 - command_output_accuracy: 0.9220 - command_output_1_accuracy: 0.0760 - participant_output_1_accuracy: 0.1630 - val_loss: 3.0534 - val_participant_output_loss: 1.1136 - val_command_output_loss: 1.9397 - val_command_output_1_loss: 4.4715e-05 - val_participant_output_1_loss: 2.4454e-05 - val_participant_output_accuracy: 0.7760 - val_command_output_accuracy: 0.9600 - val_command_output_1_accuracy: 0.0100 - val_participant_output_1_accuracy: 0.2160
Epoch 5/10
16/16 [==============================] - 3s 165ms/step - loss: 2.8495 - participant_output_loss: 0.9493 - command_output_loss: 1.9002 - command_output_1_loss: 5.0405e-05 - participant_output_1_loss: 2.3210e-05 - participant_output_accuracy: 0.9050 - command_output_accuracy: 0.9800

16/16 [==============================] - 2s 155ms/step - loss: 1.4287 - participant_output_loss: 0.2827 - command_output_loss: 1.1459 - command_output_1_loss: 7.6489e-05 - participant_output_1_loss: 1.6435e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0020 - participant_output_1_accuracy: 0.1450 - val_loss: 1.5571 - val_participant_output_loss: 0.4073 - val_command_output_loss: 1.1497 - val_command_output_1_loss: 3.4685e-05 - val_participant_output_1_loss: 2.0145e-05 - val_participant_output_accuracy: 0.9720 - val_command_output_accuracy: 0.9940 - val_command_output_1_accuracy: 0.0500 - val_participant_output_1_accuracy: 0.1640
Epoch 6/10
16/16 [==============================] - 2s 153ms/step - loss: 1.3311 - participant_output_loss: 0.2555 - command_output_loss: 1.0756 - command_output_1_loss: 2.9056e-05 - participant_output_1_loss: 1.6214e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_

Epoch 7/10
16/16 [==============================] - 2s 143ms/step - loss: 2.1540 - participant_output_loss: 0.6040 - command_output_loss: 1.5500 - command_output_1_loss: 4.8425e-05 - participant_output_1_loss: 1.2925e-05 - participant_output_accuracy: 0.9690 - command_output_accuracy: 0.9660 - command_output_1_accuracy: 0.0170 - participant_output_1_accuracy: 0.2060 - val_loss: 2.2006 - val_participant_output_loss: 0.6725 - val_command_output_loss: 1.5280 - val_command_output_1_loss: 5.0184e-05 - val_participant_output_1_loss: 1.3158e-05 - val_participant_output_accuracy: 0.9200 - val_command_output_accuracy: 0.9520 - val_command_output_1_accuracy: 0.1260 - val_participant_output_1_accuracy: 0.2220
Epoch 8/10
16/16 [==============================] - 2s 153ms/step - loss: 1.9586 - participant_output_loss: 0.5027 - command_output_loss: 1.4558 - command_output_1_loss: 8.6809e-05 - participant_output_1_loss: 1.1412e-05 - participant_output_accuracy: 0.9870 - command_output_accuracy: 0.9940

...

Epoch 3/10
16/16 [==============================] - 2s 124ms/step - loss: 1.9330 - participant_output_loss: 0.1934 - command_output_loss: 1.7396 - command_output_1_loss: 3.8829e-05 - participant_output_1_loss: 9.0615e-06 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9990 - command_output_1_accuracy: 0.5720 - participant_output_1_accuracy: 0.2490 - val_loss: 2.0609 - val_participant_output_loss: 0.3267 - val_command_output_loss: 1.7342 - val_command_output_1_loss: 8.7069e-06 - val_participant_output_1_loss: 1.0665e-05 - val_participant_output_accuracy: 0.9660 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.1700 - val_participant_output_1_accuracy: 0.2140
Epoch 4/10
16/16 [==============================] - 2s 144ms/step - loss: 1.8664 - participant_output_loss: 0.1712 - command_output_loss: 1.6952 - command_output_1_loss: 5.2309e-05 - participant_output_1_loss: 8.8170e-06 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 6/10
16/16 [==============================] - 2s 144ms/step - loss: 2.6265 - participant_output_loss: 0.7123 - command_output_loss: 1.9141 - command_output_1_loss: 7.1435e-05 - participant_output_1_loss: 2.7124e-05 - participant_output_accuracy: 0.9590 - command_output_accuracy: 0.9800 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2510 - val_loss: 2.6423 - val_participant_output_loss: 0.7530 - val_command_output_loss: 1.8893 - val_command_output_1_loss: 3.9298e-05 - val_participant_output_1_loss: 2.7619e-05 - val_participant_output_accuracy: 0.9120 - val_command_output_accuracy: 0.9820 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1560
Epoch 7/10
16/16 [==============================] - 2s 122ms/step - loss: 2.4649 - participant_output_loss: 0.6153 - command_output_loss: 1.8495 - command_output_1_loss: 3.6295e-05 - participant_output_1_loss: 2.6378e-05 - participant_output_accuracy: 0.9750 - command_output_accuracy

16/16 [==============================] - 2s 127ms/step - loss: 1.4878 - participant_output_loss: 0.2193 - command_output_loss: 1.2683 - command_output_1_loss: 9.6779e-05 - participant_output_1_loss: 1.9611e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2180 - val_loss: 1.6392 - val_participant_output_loss: 0.3682 - val_command_output_loss: 1.2709 - val_command_output_1_loss: 4.7179e-05 - val_participant_output_1_loss: 2.1806e-05 - val_participant_output_accuracy: 0.9540 - val_command_output_accuracy: 0.9940 - val_command_output_1_accuracy: 0.0020 - val_participant_output_1_accuracy: 0.1900
Epoch 8/10
16/16 [==============================] - 2s 153ms/step - loss: 1.4138 - participant_output_loss: 0.1992 - command_output_loss: 1.2146 - command_output_1_loss: 3.9333e-05 - participant_output_1_loss: 1.8118e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - comm

Epoch 9/10
16/16 [==============================] - 2s 127ms/step - loss: 1.8487 - participant_output_loss: 0.4862 - command_output_loss: 1.3624 - command_output_1_loss: 5.9451e-05 - participant_output_1_loss: 1.3764e-05 - participant_output_accuracy: 0.9960 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1830 - val_loss: 1.9375 - val_participant_output_loss: 0.5924 - val_command_output_loss: 1.3451 - val_command_output_1_loss: 3.2592e-05 - val_participant_output_1_loss: 1.6380e-05 - val_participant_output_accuracy: 0.9320 - val_command_output_accuracy: 0.9920 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2040
Epoch 10/10
16/16 [==============================] - 2s 119ms/step - loss: 1.6926 - participant_output_loss: 0.4282 - command_output_loss: 1.2643 - command_output_1_loss: 3.2151e-05 - participant_output_1_loss: 1.2830e-05 - participant_output_accuracy: 0.9980 - command_output_accurac

16/16 [==============================] - 2s 127ms/step - loss: 0.6636 - participant_output_loss: 0.1192 - command_output_loss: 0.5444 - command_output_1_loss: 1.9647e-05 - participant_output_1_loss: 7.1579e-06 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2060 - val_loss: 0.8220 - val_participant_output_loss: 0.2569 - val_command_output_loss: 0.5651 - val_command_output_1_loss: 1.3056e-05 - val_participant_output_1_loss: 1.2491e-05 - val_participant_output_accuracy: 0.9700 - val_command_output_accuracy: 0.9980 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1640


In [12]:
Perfomance

,Command_Acc,Feature_Extract,Inputs,Partcp_Acc,Time
0,0.982,ResNet,SPEC,0.970,53.745848
1,0.980,ResNet,SPEC,0.968,59.788192
2,0.972,ResNet,SPEC,0.960,60.039808
3,0.982,ResNet,SPEC,0.958,56.502710
4,0.980,ResNet,SPEC,0.968,56.194534
5,0.986,ResNet,SPEC,0.966,55.374021
6,0.978,ResNet,SPEC,0.952,55.079770
7,0.988,ResNet,SPEC,0.960,55.301056
8,0.984,ResNet,SPEC,0.960,51.448608
9,0.980,ResNet,SPEC,0.952,52.307214


In [13]:
Perfomance.to_csv('Performance_0915_Digit_Inputs.csv',index=False)